## Requests

O que é um **request**?
Um request é um pedido de informação, normalmente a um website que oferece um serviço. Por exemplo, se você quer saber se um CEP é válido ou a qual endereço ele se refere, existe o webservice <a href="https://viacep.com.br/" target="_blank">Viacep</a>. Por exemplo, se você entrar com o link https://viacep.com.br/ws/36570047/json/ no navegador, ele irá te retornar uma resposta no formado JSON com dados referentes ao endereço do CEP 3657-047.


Vamos começar importando a biblioteca python <a href="https://requests.readthedocs.io/en/master/" target="_blank">**requests**</a>:

In [ ]:
import requests

Feito isso, vamos criar a função `achaEndereco` que recebe um CEP e retorna um endereço como resposta:

In [ ]:
def achaEndereco(cep):
    resposta = requests.get('http://viacep.com.br/ws/' + cep + '/json/')
    resposta = resposta.json()

    return resposta

Agora, quando executamos a função enviando um CEP como parâmetro temos o endereço referente ao CEP como um dicionário python:

In [ ]:
achaEndereco('36570047')

Através do mesmo serviço também é possível fazer o caminho contrário e encontrar um CEP a partir de um determinado endereço:

In [ ]:
def achaCEP(estado, cidade, rua):
    resposta = requests.get('http://viacep.com.br/ws/' + estado + '/' + cidade + '/' + rua + '/json/')
    resposta = resposta.json()

    return resposta

Como é possível que um endereço tenha mais de um CEP (no caso de um endereço incompleto, por exemplo), o serviço retorna uma lista de possíveis endereços:

In [ ]:
achaCEP('MG', 'Viçosa', 'Avenida Bueno Brandão')

Vamos agora utilizar um outro serviço interessante, o <a href="https://www.graphhopper.com/">**GraphHopper**</a>. Com o GraphHopper é possível encontrar as coordenadas de um local a partir de uma busca pelo endereço, um recurso conhecido como _Geocoding_, entre outros recursos. Antes de tudo é necessário termos uma conta no site pois o serviçosa requer uma chave de desenvolvedor. Para as finalidades deste exercício, usarei minha própria chave. Para facilitar o uso da chave do código, vamos declará-la no código como uma constante:

In [ ]:
GRAPHHOPPER_API_KEY = 'f9e77954-a5fd-416e-8087-1c1c456c6fc8'

Vamos agora criar a função `achaCoord`, que recebe como parâmetro uma string de busca (um endereço) e retorna a coordenada mais provável para este endereço:

In [53]:
def achaCoord(endereco):
    locale = 'br'
    url = 'https://graphhopper.com/api/1/geocode?q=' + endereco + '&locale=' + locale + '&key=' + GRAPHHOPPER_API_KEY
    resposta = requests.get(url)
    resposta = resposta.json()

    return (resposta['hits'][0]['point']['lat'], resposta['hits'][0]['point']['lng'])

Para entendermos como funciona essa função é preciso dar uma olhada na <a href="https://docs.graphhopper.com/#tag/Geocoding-API" target="_blank">documentação</a> do serviço.
A URL, ou seja, o endereço web do serviço, é sempre 'https://graphhopper.com/api/1/geocode?', seguida de seus parâmetros, separados por '&': `q=query_de_busca`, `locale=br` para endereços localizados no Brasil e `key=chave`. O serviço retorna um JSON (convertido para um dicionário em python) com as chaves _hits_, que é uma lista de prováveis coordenadas, e _took_, que é o tempo que o serviço demorou para encontrar os resultados. Cada _hit_ encontrado tem é formado de diversos dados sobre o endereço e, entre eles, um ponto, que é um par de coordenadas (lat, lng), que é o que a função retorna:

In [56]:
casa = achaCoord('Avenida Bueno Brandão, Viçosa - MG')

In [ ]:
casa

Agora vamos um pouquinho mais longe e usar uma outra biblioteca python para encontrar essa coordenada em um mapa. A biblioteca <a href="https://python-visualization.github.io/folium/" target="_blank">**Folium**</a> pode nos auxiliar nisso:

In [ ]:
import folium

A biblioteca **Folium** é um encapsulamento para que a biblioteca Javascript <a href="https://leafletjs.com/" target="_blank">**Leaflet**</a> possa ser usada em python. É uma ferramenta que permite utilizar diversos serviços de mapas, ou seja, você pode utilizar o serviços livre <a href="https://www.openstreetmap.org/" target="_blank">**OpenStreetMap** (OSM)</a> para desenhar seu mapa, mas também pode utilizar serviços pagos como <a href="https://www.mapbox.com/" target="_blank">**Mapbox**</a>. Neste exemplo vamos utilizar o OSM, que é o padrão.

Para criar um mapa com o Folium basta uma linha de código:

In [ ]:
m = folium.Map(location=casa, zoom_start=15)

E com isso temos:

In [57]:
m

In [ ]:
folium.Marker(coord, icon=folium.Icon(icon='home', color='blue', prefix='fa')).add_to(m)